In [1]:
import os
import json
from PIL import Image

In [2]:
items = [f[:-4] for f in os.listdir('Raw') if f[-4:]=='.bmp']
len(items)

509

In [3]:
# 1. bmp2jpg
for item in items:
    im = Image.open('Raw/'+item+'.bmp')
    rgb_im = im.convert('RGB')
    rgb_im.save('Raw_Images_JPG/'+item+'.jpg')

In [4]:
# 2. bounding box to Pascal VOC format

In [5]:
# check if table bbox exists
valid_items = []
for item in items:
    with open('Labeled/'+item+'.xml', "r") as f:
        xml = f.read()
        if xml.find('Composites Label="Table"')!=-1:
            valid_items.append(item)
            
print(len(valid_items))

478


In [6]:
import struct
import binascii

for item in valid_items:
    width, height = Image.open('Raw_Images_JPG/'+item+'.jpg').size
    template = '''
<annotation>
  <filename>%(item)s.jpg</filename>
  <size>
    <width>%(width)s</width>
    <height>%(height)s</height>
    <depth>3</depth>
  </size>
  <segmented>0</segmented>'''%{
    'item': item,
    'width': width,
    'height': height,}

    with open('Labeled/'+item+'.xml', "r") as f:
        xml = f.read()
        table_block = xml.split('Composites Label="Table"')[1].split('</Composites>')[0] # store all tables
        samples = table_block.split('<Composite BBox="')[1:]
        for sample in samples:
            bbox_hex = sample.split('"')[0].split() # wannable bbox (hex string)
            bbox_int = [round(96/72*struct.unpack('!d', bytes.fromhex(x))[0]) for x in bbox_hex] #from ieee754 to int
            xmin, ymin, xmax, ymax = bbox_int
            ymin = height - ymin
            ymax = height - ymax
            assert width>=xmin and width>=xmax and height>=ymin and height>=ymax
            assert xmax>=xmin and ymax>=ymin
            template += '''
  <object>
    <name>table</name>
    <bndbox>
      <xmin>%(xmin)s</xmin>
      <ymin>%(ymin)s</ymin>
      <xmax>%(xmax)s</xmax>
      <ymax>%(ymax)s</ymax>
    </bndbox>
  </object>
  '''%{
    'xmin': min(xmin, xmax), # all due to shitty xmls
    'ymin': min(ymin, ymax),
    'xmax': max(xmin, xmax),
    'ymax': max(ymin, ymax),
    }
    template += '</annotation>'
    with open('Annotations/'+item+'.xml', 'w') as outfile:
        outfile.write(template)

In [7]:
with open('trainval.txt', 'w') as outfile:
    outfile.write('\n'.join(valid_items))

In [8]:
invalid_items = [x for x in items if x not in valid_items]

In [ ]:
for item in valid_items:
    im = Image.open('Raw/'+item+'.bmp')
    rgb_im = im.convert('RGB')
    rgb_im.save('Valid_Images/'+item+'.jpg')
    
for item in invalid_items:
    im = Image.open('Raw/'+item+'.bmp')
    rgb_im = im.convert('RGB')
    rgb_im.save('invalid_Images/'+item+'.jpg')

In [74]:
from PIL import Image, ImageDraw
import struct
import binascii
import matplotlib.pyplot as plt
%matplotlib inline

i=112
for item in valid_items[i:i+1]:
    width, height = Image.open('Raw_Images_JPG/'+item+'.jpg').size
    im = Image.open('Raw_Images_JPG/'+item+'.jpg')
    with open('Labeled/'+item+'.xml', "r") as f:
        xml = f.read()
        table_block = xml.split('Composites Label="Table"')[1].split('</Composites>')[0] # store all tables
        samples = table_block.split('<Composite BBox="')[1:]
        for sample in samples:
            bbox_hex = sample.split('"')[0].split() # wannable bbox (hex string)
            bbox_int = [round(96/72*struct.unpack('!d', bytes.fromhex(x))[0]) for x in bbox_hex] #from ieee754 to int
            xmin, ymin, xmax, ymax = bbox_int
            ymin = height - ymin
            ymax = height - ymax
            print(xmin, ymin, xmax, ymax)
            draw = ImageDraw.Draw(im)
            draw.line((xmin, ymin) + (xmin, ymax), fill=128)
            draw.line((xmin, ymin) + (xmax, ymin), fill=128)
            draw.line((xmax, ymax) + (xmin, ymax), fill=128)
            draw.line((xmax, ymax) + (xmax, ymin), fill=128)
            del draw
    im.show()

245 657 559 741
179 324 542 436
